# Testing photom step with MIRI simulated data

## Summary

This notebook processes an image through calwebb_image2 and calwebb_image3 (calwebb_detector1 is optional) and examines the output table of the source_catalog step. The steps are as follow:

1) Set up data path and directory and image file name.

2) Modify header information of input simulations (if needed).

3) Run input data through calwebb_detector1.

4) Run output of calwebb_detector1 through the photom step in calwebb_image2.

5) Get photom reference file. 

6) Compare the flat field reference file with the rate/cal image ratio and check that the correct conversion factor was applied.

7) Check that the pixel area information has been attached and that PIXAR_SR, PIXAR_A2, and BUNIT keywords have been populated in the cal image.


The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

Author: T. Temim

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)

### Set up import statements

In [2]:
import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from astropy.io import fits
import numpy as np
from jwst.datamodels import RampModel, ImageModel, MirImgPhotomModel
from astropy.utils.data import get_pkg_data_filename
from ci_watson.artifactory_helpers import get_bigdata
from astropy import table
import crds

2021-04-02 19:51:28,438 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/photutils/detection/findstars.py:33: AstropyDeprecationWarning: _StarFinderKernel was moved to the photutils.detection._utils module. Please update your import statement.
  warnings.warn(f'{name} was moved to the {deprecated[name]} module. '



In [3]:
#import os
#os.environ['CRDS_PATH']='$HOME/crds_cache'
#os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
#os.environ['CRDS_CONTEXT']='jwst_0619.pmap'
#os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'
#export CRDS_SERVER_URL=https://jwst-crds.stsci.edu
#export CRDS_PATH=$HOME/crds_cache
#export CRDS_CONTEXT='jwst_0619.pmap'
#export TEST_BIGDATA=https://bytesalad.stsci.edu/artifactory/

### Print pipeline version number

In [4]:
jwst.__version__ 

'1.1.0'

### Read in data from artifactory

In [5]:
input_file = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'source_catalog',
                     'source_catalog_miri_test', 
                     'det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits')

### Read in input image as JWST data model

In [6]:
from jwst import datamodels

im = ImageModel(input_file)

### Modify header information of input simulations (if needed)

In [7]:
print(im.meta.wcsinfo.wcsaxes)

4


In [8]:
im.meta.wcsinfo.wcsaxes=2

In [9]:
print(im.meta.wcsinfo.wcsaxes)

2


In [10]:
del im.meta.wcsinfo.cdelt3
del im.meta.wcsinfo.crpix3
del im.meta.wcsinfo.crval3
del im.meta.wcsinfo.ctype3
del im.meta.wcsinfo.cunit3
del im.meta.wcsinfo.pc3_1
del im.meta.wcsinfo.pc3_2

### Run input data through calwebb_detector1

In [11]:
# The data read in is an image file already processed through Detector1. This step should only be run 
#if we're starting with a ramp file.

#det1 = Detector1Pipeline()
#det1.save_results = True
#det1.run(im)

### Run output of calwebb_detector1 through calwebb_image2

In [12]:
#input_file = input_file.replace('rateint.fits', 'rate.fits')

In [13]:
im2 = Image2Pipeline()
#im2.background.skip = True
im2.assign_wcs.skip = True
im2.flat_field.skip = True
im2.photom.skip=False
im2.resample.skip = True
im2.save_results = True
im2.run(im)

2021-04-02 19:51:29,019 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.


2021-04-02 19:51:29,020 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.


2021-04-02 19:51:29,022 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.


2021-04-02 19:51:29,023 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.


2021-04-02 19:51:29,024 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.


...

2021-04-02 19:51:30,939 - stpipe.Image2Pipeline - INFO - Finished processing product det_image_seq1_MIRIMAGE_F560Wexp1_rate


2021-04-02 19:51:30,940 - stpipe.Image2Pipeline - INFO - ... ending calwebb_image2


2021-04-02 19:51:31,015 - stpipe.Image2Pipeline - INFO - Saved model in det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits


2021-04-02 19:51:31,016 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline done


[<ImageModel(1024, 1032) from det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits>]

In [14]:
input_file = input_file.replace('rate.fits', 'cal.fits')

In [15]:
# read in the calibrated file
im_cal = ImageModel(input_file)

### Calculate the rate/cal image ratio

In [16]:
ratio_im=im_cal.data/im.data

2021-04-02 19:51:31,078 - stpipe - WARNING - /data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.8/site-packages/asdf/tags/core/ndarray.py:556: RuntimeWarning: invalid value encountered in true_divide
  return getattr(self._make_array(), name)(*args)



In [17]:
print(ratio_im)

[[1.12701   1.12701   1.12701   ... 1.12701   1.12701   1.12701  ]
 [1.12701   1.12701   1.12701   ... 1.12701   1.12701   1.12701  ]
 [1.12701   1.12701   1.12701   ... 1.1270099 1.12701   1.12701  ]
 ...
 [1.12701   1.12701   1.12701   ... 1.12701   1.12701   1.12701  ]
 [      nan       nan       nan ... 1.12701   1.12701   1.12701  ]
 [1.12701   1.12701   1.12701   ... 1.12701   1.12701   1.12701  ]]


### Get photom reference file

In [18]:
photomreffile = im_cal.meta.ref_file.photom.name
print('Photom reference file', photomreffile)
# find location of file
basename = crds.core.config.pop_crds_uri(photomreffile)
path = crds.locate_file(basename, "jwst")
# open reference file

photom_file = MirImgPhotomModel(path)

Photom reference file crds://jwst_miri_photom_0073.fits


In [19]:
photom_file.meta.photometry.pixelarea_steradians

2.84403609523084e-13

### Print photom calibration conversion factor for image

In [20]:
print(photom_file.phot_table['photmjsr'][np.where(photom_file.phot_table['filter'] == im_cal.meta.instrument.filter)][0])

1.12701


### Check that same value has been applied to cal image (three values should be the same)

In [21]:
print('image min','image max','ref file')
print(np.nanmin(ratio_im),np.nanmax(ratio_im),photom_file.phot_table['photmjsr'][np.where(photom_file.phot_table['filter'] == im_cal.meta.instrument.filter)][0])

image min image max ref file
1.1270099 1.1270101 1.12701


### Check that pixel area information has been attached

In [22]:
im_cal.area
np.nanmin(im_cal.area)
np.nanmax(im_cal.area)

1.0159259

In [23]:
print('pix_area_min', 'pix_area_max','pix_area_mean')
print(np.nanmin(im_cal.area),np.nanmax(im_cal.area),np.nanmean(im_cal.area))

pix_area_min pix_area_max pix_area_mean
0.97582465 1.0159259 1.0058197


### Check that PIXAR_SR and PIXAR_A2 keywords have been populated

In [24]:
im_cal.find_fits_keyword('PIXAR_SR')

['meta.photometry.pixelarea_steradians']

In [25]:
photom_file.find_fits_keyword('PIXAR_SR')

['meta.photometry.pixelarea_steradians']

#### Two values should be identical

In [26]:
print(photom_file.meta.photometry.pixelarea_steradians,im_cal.meta.photometry.pixelarea_steradians)

2.84403609523084e-13 2.84403609523084e-13


#### Two values should be identical

In [27]:
print(photom_file.meta.photometry.pixelarea_arcsecsq, im_cal.meta.photometry.pixelarea_arcsecsq)

0.0121 0.0121


### Verify that pixel area in steradians is correct (values should be the same)

In [28]:
print(im_cal.meta.photometry.pixelarea_arcsecsq/(206265.0**2),im_cal.meta.photometry.pixelarea_steradians)

2.844030752201317e-13 2.84403609523084e-13


### Check that bunit information has been added to image ('MJy/sr')

In [29]:
im_cal.meta.bunit_data

'MJy/sr'

In [30]:
#photom_file.meta.author